In [ ]:
import pathlib
import os
import random
import jupedsim as jps
import pedpy
from matplotlib.patches import Circle
from shapely import GeometryCollection, Point, Polygon
from fdsvismap import VisMap
import shapely
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## FDS geometry
```
&OBST XB=0.0,19.0, 5.0,5.1, 0.0,3.0 SURF_ID='WALL' /
&OBST XB=4.0,4.1, 8.0,15.0, 0.0,3.0 SURF_ID='WALL' /
&OBST XB=4.1,18.9, 8.0,8.1, 0.0,3.0 SURF_ID='WALL' /
&OBST XB=18.9,19.0, 0.0,15.0, 0.0,5.0 SURF_ID='WALL' /
&OBST XB=15.0,15.1, 0.0,5.0, 0.0,3.0 SURF_ID='WALL' /
&OBST XB=0.0,19.0, 0.0,15.0, 3.0,3.1 SURF_ID='WALL' /

&HOLE XB=18.9,19.0, 6.0,7.2, 0.0,2.0 /
&HOLE XB=18.9,19.0, 1.0,2.0, 0.0,2.0 /

X0,X1, Y0,Y1, Z0,Z1
```

In [ ]:
def plot_data_2d(data):
    fig, ax = plt.subplots()
    def parse_coords(coord_str):
        coords = coord_str.replace('XB=', '').split(',')
        return list(map(float, coords))

    for line in data:
        if 'XB=' in line:
            coord_part = line.split(' ')[1]
            xb = parse_coords(coord_part)
            if len(xb) >= 4:
                rect = patches.Rectangle((xb[0], xb[2]), xb[1]-xb[0], xb[3]-xb[2],
                                         linewidth=1, edgecolor='black',
                                         facecolor='white' if 'OBST' in line else 'blue', alpha=0.6)
                ax.add_patch(rect)

    ax.set_xlim(0, 20)
    ax.set_ylim(0, 16)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('2D Visualization of Obstacles and Holes')
    plt.grid(True)
    plt.show()

data = [
    "&OBST XB=0.0,19.0,5.0,5.1,0.0,3.0 SURF_ID='WALL' /",
    "&OBST XB=4.0,4.1,8.0,15.0,0.0,3.0 SURF_ID='WALL' /",
    "&OBST XB=4.1,18.9,8.0,8.1,0.0,3.0 SURF_ID='WALL' /",
    "&OBST XB=18.9,19.0,0.0,15.0,0.0,5.0 SURF_ID='WALL' /",
    "&OBST XB=15.0,15.1,0.0,5.0,0.0,3.0 SURF_ID='WALL' /",
    "&OBST XB=0.0,19.0,0.0,15.0,3.0,3.1 SURF_ID='WALL' /",
    "&HOLE XB=18.9,19.0,6.0,7.2,0.0,2.0 /",
    "&HOLE XB=18.9,19.0,1.0,2.0,0.0,2.0 /"
]

plot_data_2d(data)


In [ ]:
def read_wkt_from_file(filename):
    with open(filename, 'r') as file:
        wkt_data = file.readlines()
    return wkt_data

def data_to_wkt(data):
    # List to hold WKT strings
    wkt_polygons = []

    # Function to parse coordinates and format as WKT POLYGON
    def format_wkt(xb):
        # Format the coordinates as a WKT POLYGON string
        x0, x1, y0, y1 = xb[:4]
        return f"POLYGON(({x0} {y0}, {x1} {y0}, {x1} {y1}, {x0} {y1}, {x0} {y0}))"

    # Parse the coordinate string
    def parse_coords(coord_str):
        coords = coord_str.replace('XB=', '').split(',')
        return list(map(float, coords))

    # Extract and format each line's data
    for line in data:
        if 'XB=' in line:
            # Extract the part containing the coordinates
            coord_part = line.split(' ')[1]
            # Parse coordinates
            xb = parse_coords(coord_part)
            # Format as WKT
            if len(xb) >= 4:
                wkt_polygon = format_wkt(xb)
                wkt_polygons.append(wkt_polygon)

    return wkt_polygons

def save_wkt_to_file(wkt_polygons, filename):
    with open(filename, 'w') as file:
        for polygon in wkt_polygons:
            file.write(polygon + '\n')

data = [
    "&OBST XB=0.0,19.0,5.0,5.1,0.0,3.0 SURF_ID='WALL' /",
    "&OBST XB=4.0,4.1,8.0,15.0,0.0,3.0 SURF_ID='WALL' /",
    "&OBST XB=4.1,18.9,8.0,8.1,0.0,3.0 SURF_ID='WALL' /",
    "&OBST XB=18.9,19.0,0.0,15.0,0.0,5.0 SURF_ID='WALL' /",
    "&OBST XB=15.0,15.1,0.0,5.0,0.0,3.0 SURF_ID='WALL' /",
    "&OBST XB=0.0,19.0,0.0,15.0,3.0,3.1 SURF_ID='WALL' /",
    "&HOLE XB=18.9,19.0,6.0,7.2,0.0,2.0 /",
    "&HOLE XB=18.9,19.0,1.0,2.0,0.0,2.0 /"
]

# Convert data to WKT
wkt_polygons = data_to_wkt(data)

# Save to WKT file
save_wkt_to_file(wkt_polygons, 'output.wkt')

print("WKT data has been saved to 'output.wkt'.")
wkt_data = read_wkt_from_file('output.wkt')

## Vismap config

In [ ]:
project_root = pathlib.Path(os.path.abspath('')) / "examples" / "room_fire"
bg_img = project_root  /"misc" / "floorplan.png"  # Create instance of VisMap class

# Set path for FDS simulation directory and background image
sim_dir = str(project_root / "fds_data")

# Create instance of VisMap class
vis = VisMap()

# Read data from FDS simulation directory
vis.read_fds_data(sim_dir)
vis.add_background_image(bg_img)

In [ ]:
# Set starpoint and waypoints along escape route
vis.set_start_point(8, 8)
vis.set_waypoint(x=8, y=5, c=3, alpha=180)
vis.set_waypoint(x=9.8, y=6, c=3, alpha=270)
vis.set_waypoint(x=17, y=0, c=3, alpha=0)

# Set times when the simulation should be evaluated
times = range(0, 500, 50)
vis.set_time_points(times)

# Do the required calculations to create the Vismap
vis.compute_all()

In [ ]:
fig, ax = vis.create_aset_map_plot()
ax.set_xlim(0, 20)
ax.set_ylim(10, 0)

In [ ]:
fig, ax = vis.create_time_agg_wp_agg_vismap()
ax.set_xlim(0, 20)
ax.set_ylim(10, 0)

# Check if waypoint is visible from given location at given time
print(vis.wp_is_visible(time=50, x=12.5, y=0.6, waypoint_id=2))

# Get distance from waypoint to given location
print(f"{vis.get_distance_to_wp(x=17, y=5, waypoint_id=2)=}")

# Calculate local visibility at given location and time, considering a specific c factor
print(vis.get_local_visibility(time=100, x=5, y=6, c=3))


## Definition of Start Positions and Exit

Now we define the spawning area and way points for the leader to follow.

In [ ]:
area = Polygon([(0, 0), (30, 0), (30, 20), (0, 20)])
obstacles = [
Polygon([(0, 5), (10, 5), (10, 5.1), (0, 5.1)]),
Polygon([(11, 5), (19, 5), (19, 5.1), (11, 5.1)]),

Polygon([(4, 8), (4.1, 8), (4.1, 15), (4, 15)]),

Polygon([(0, 15), (2.0, 15), (2.0, 15.1), (0, 15.1)]),
Polygon([(3.0, 15), (4.1, 15), (4.1, 15.1), (3.0, 15.1)]),

Polygon([(4.1, 15), (19, 15), (19, 15.1), (4.1, 15.1)]),

Polygon([(4.1, 8), (13, 8), (13, 8.1), (4.1, 8.1)]),
Polygon([(14, 8), (18.89, 8), (18.89, 8.1), (14, 8.1)]),

Polygon([(18.9, 5.0), (19.0, 5.0), (19.0, 6), (18.9, 6), (18.9, 5.0)]),
Polygon([(18.9, 0), (18.9, 5), (19.0, 5), (19, 0), (18.9,0)]),

Polygon([(18.9, 7.0), (19.0, 7.0), (19.0, 15.1), (18.9, 15.1), (18.9, 7.0)]),
Polygon([(15, 0), (15.1, 0), (15.1, 4.99), (15, 4.99)]),
]
obstacle = shapely.union_all(obstacles)
walkable_area = pedpy.WalkableArea(shapely.difference(area, obstacle))
pedpy.plot_walkable_area(walkable_area=walkable_area)

In [ ]:
area = Polygon([(0, 0), (30, 0), (30, 20), (0, 20)])

obstacles = [
Polygon([(0, 15), (2.0, 15), (2.0, 15.1), (0, 15.1)]),
Polygon([(0, 5), (10, 5), (10, 5.1), (0, 5.1)]),

Polygon([(15, 0), (15, 5), (11, 5), (11, 5.1), (18.9, 5.1), (18.9, 6), (19, 6), (19, 0)]),
#Polygon([(18.9, 0), (18.9, 5.0), (15.1, 5.0), (15.1, 0)]),

Polygon([(4.0, 8), (13, 8), (13, 8.1), (4.1, 8.1), (4.1, 15.0), (18.9, 15.0), (18.9, 8.1), (14.0, 8.1), (14.0, 8.0), (18.9, 8.0), (18.9, 7.0), (19, 7.0), (19, 15.1), (3, 15.1), (3, 15.0), (4.0, 15.0)]),
]
obstacle = shapely.union_all(obstacles)
walkable_area = pedpy.WalkableArea(shapely.difference(area, obstacle))
pedpy.plot_walkable_area(walkable_area=walkable_area)

In [ ]:
num_agents = 3
seed = 1
spawning_area1 = Polygon([(0, 0), (0, 5), (5, 5), (5, 0)])
spawning_area2 = Polygon([(4.1, 10), (7, 10), (7, 14.5), (4.1, 14.5)])
pos_in_spawning_areas = [
    jps.distributions.distribute_by_number(
    polygon=spawning_area2,
    number_of_agents=num_agents,
    distance_to_agents=0.4,
    distance_to_polygon=0.15,
    seed=seed,
),
  jps.distributions.distribute_by_number(
    polygon=spawning_area1,
    number_of_agents=num_agents,
    distance_to_agents=0.4,
    distance_to_polygon=0.15,
    seed=seed,
)
]
exits = [
    Polygon([(2, 15.5), (3, 15.5), (3, 16.5), (2, 16.5), (2, 15.5)]),
    Polygon([(28, 15.5), (29, 15.5), (29, 16.5), (28, 16.5), (28, 15.5)]),
]   
waypoints = [
    (13.5, 8),
    (10.5, 5),
    (19, 6.5),
]
distance_to_waypoints = 0.5

In [ ]:
def plot_simulation_configuration(
    walkable_area, starting_positions, exits
):
    axes = pedpy.plot_walkable_area(walkable_area=walkable_area)
    for exit_area in exits:
        axes.fill(*exit_area.exterior.xy, color="indianred")

    for starting_position in starting_positions:
      
        axes.scatter(*zip(*starting_position), s=1, color='gray')    

    axes.set_xlabel("x/m")
    axes.set_ylabel("y/m")
    axes.set_aspect("equal")
    for idx, waypoint in enumerate(waypoints):
        axes.plot(waypoint[0], waypoint[1], "ro")
        axes.annotate(
            f"WP {idx+1}",
            (waypoint[0], waypoint[1]),
            textcoords="offset points",
            xytext=(10, -15),
            ha="center",
        )
        circle = Circle(
            (waypoint[0], waypoint[1]),
            distance_to_waypoints,
            fc="red",
            ec="red",
            alpha=0.1,
        )
        axes.add_patch(circle)


plot_simulation_configuration(
    walkable_area, pos_in_spawning_areas, exits
)

## Specification of Parameters und Running the Simulation

Now we just need to define the details of the operational model as well as the exit.

In [ ]:
trajectory_file = "output.sqlite"  # output file
simulation = jps.Simulation(
    model=jps.GeneralizedCentrifugalForceModel(
        max_neighbor_repulsion_force=10,
        max_geometry_repulsion_force=10,
        max_neighbor_interaction_distance=2,
        max_neighbor_interpolation_distance=0.1,
        strength_neighbor_repulsion=0.3,
    ),
    geometry=walkable_area.polygon,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)
exit_ids = [simulation.add_exit_stage(exit_area.exterior.coords[:-1]) for exit_area in exits]

## Define Journey for people in the upper room going right


In [ ]:
waypoint_ids = [
    simulation.add_waypoint_stage(waypoint, distance_to_waypoints)
    for waypoint in waypoints
]
journey_up_right= jps.JourneyDescription([*waypoint_ids, *exit_ids])
journey_up_right.set_transition_for_stage(
        waypoint_ids[0],
        jps.Transition.create_fixed_transition(
            waypoint_ids[2]),
    )
journey_up_right.set_transition_for_stage(
        waypoint_ids[2],
        jps.Transition.create_fixed_transition(
            exit_ids[1]),
    )
journey_up_right_id = simulation.add_journey(journey_up_right)

## Define Journey for people in the upper room going left


In [ ]:
journey_up_left= jps.JourneyDescription([*waypoint_ids, *exit_ids])
journey_up_left.set_transition_for_stage(
        waypoint_ids[0],
        jps.Transition.create_fixed_transition(
            exit_ids[0]),
    )
journey_up_left_id = simulation.add_journey(journey_up_left)

## Define Journey for people in the lower room going right

In [ ]:
journey_down_right= jps.JourneyDescription([*waypoint_ids, *exit_ids])
journey_down_right.set_transition_for_stage(
        waypoint_ids[1],
        jps.Transition.create_fixed_transition(
            waypoint_ids[2]),
    )
journey_down_right.set_transition_for_stage(
        waypoint_ids[2],
        jps.Transition.create_fixed_transition(
            exit_ids[1]),
    )
journey_down_right_id = simulation.add_journey(journey_down_right)

## Define Journey for people in the lower room going left

In [ ]:
journey_down_left= jps.JourneyDescription([*waypoint_ids, *exit_ids])
journey_down_left.set_transition_for_stage(
        waypoint_ids[1],
        jps.Transition.create_fixed_transition(
            exit_ids[0]),
    )
journey_down_left_id = simulation.add_journey(journey_down_left)

## Add agents

First, add leader, then its followers.

In [ ]:
ids_up = set(
    [
        simulation.add_agent(
            jps.GeneralizedCentrifugalForceModelAgentParameters(
                journey_id=journey_up_right_id,
                stage_id=waypoint_ids[0],
                position=pos,
                v0=0.8,
                b_min=0.1,
                b_max=0.2,
                a_min=0.1,
                a_v=0.2,
                orientation=(1, 0),
            )
        )
        for pos in pos_in_spawning_areas[0]
    ]
)
ids_down = set(
    [
        simulation.add_agent(
            jps.GeneralizedCentrifugalForceModelAgentParameters(
                journey_id=journey_down_right_id,
                stage_id=waypoint_ids[1],
                position=pos,
                v0=0.8,
                b_min=0.1,
                b_max=0.2,
                a_min=0.1,
                a_v=0.2,
                orientation=(1, 0),
            )
        )
        for pos in pos_in_spawning_areas[1]
    ]
)

## Simulation loop

In [ ]:
while simulation.agent_count() > 0:
    for agent in simulation.agents():
        condition = simulation.iteration_count() > 300
        if agent.journey_id == journey_up_right_id and condition:
            simulation.switch_agent_journey(agent.id, journey_up_left_id, exit_ids[0])
        if agent.journey_id == journey_down_right_id and condition:
            simulation.switch_agent_journey(agent.id, journey_down_left_id, exit_ids[0])

    simulation.iterate()

## Visualization

Let's have a look at the visualization of the simulated trajectories:

In [ ]:
from jupedsim.internal.notebook_utils import animate, read_sqlite_file

trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
animate(trajectory_data, walkable_area, every_nth_frame=10)